In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'gender_submission.csv']


In [2]:
data=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [4]:
data.Age.fillna(data.Age.median(),inplace=True)
test.Age.fillna(data.Age.median(),inplace=True)
test.Fare.fillna(data['Fare'][data['Pclass']==3].median(),inplace=True)
data.Embarked.fillna('S',inplace=True)
data.drop(['Cabin','PassengerId','Ticket','Name'],axis=1,inplace=True)
test.drop(['Cabin','PassengerId','Ticket','Name'],axis=1,inplace=True)
data.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
Pclass      418 non-null int64
Sex         418 non-null object
Age         418 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        418 non-null float64
Embarked    418 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 22.9+ KB


In [5]:
dummies=pd.get_dummies(data['Sex'])
data=pd.concat([data,dummies],axis=1)
data.drop('Sex',axis=1,inplace=True)
data.info()
dummies=pd.get_dummies(test['Sex'])
test=pd.concat([test,dummies],axis=1)
test.drop('Sex',axis=1,inplace=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null object
female      891 non-null uint8
male        891 non-null uint8
dtypes: float64(2), int64(4), object(1), uint8(2)
memory usage: 50.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
Pclass      418 non-null int64
Age         418 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        418 non-null float64
Embarked    418 non-null object
female      418 non-null uint8
male        418 non-null uint8
dtypes: float64(2), int64(3), object(1), uint8(2)
memory usage: 20.5+ KB


In [6]:
dummies=pd.get_dummies(data['Embarked'])
data=pd.concat([data,dummies],axis=1)
data.drop('Embarked',axis=1,inplace=True)
data.info()
dummies=pd.get_dummies(test['Embarked'])
test=pd.concat([test,dummies],axis=1)
test.drop('Embarked',axis=1,inplace=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
female      891 non-null uint8
male        891 non-null uint8
C           891 non-null uint8
Q           891 non-null uint8
S           891 non-null uint8
dtypes: float64(2), int64(4), uint8(5)
memory usage: 46.2 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
Pclass    418 non-null int64
Age       418 non-null float64
SibSp     418 non-null int64
Parch     418 non-null int64
Fare      418 non-null float64
female    418 non-null uint8
male      418 non-null uint8
C         418 non-null uint8
Q         418 non-null uint8
S         418 non-null uint8
dtypes: float64(2), int64(3), uint8(5)
memory usage: 18.4 KB


In [7]:
data.Age=data.Age.astype(int)

In [8]:
import xgboost as xgb
model=xgb.XGBClassifier(learning_rate=0.001,n_estimators=1000)
model.fit(data.drop('Survived',axis=1),data.Survived)
print(model.score(data.drop('Survived',axis=1),data.Survived))

0.8316498316498316


In [9]:
ans=model.predict(test)
x=pd.read_csv('../input/gender_submission.csv')
a=pd.DataFrame({'PassengerId':x['PassengerId'],'Survived':ans})
a.to_csv('submit.csv',index=False)